# Homework3
顾淳

19307110344

In [1]:
# --- Explore HMM POS Taggers using Brown corpus ---

# In this assignment, you will explore two taggers for a Brown corpus.

# import your packages here

In [2]:
import nltk
import numpy as np
import pandas as pd
from nltk.probability import ConditionalFreqDist,ConditionalProbDist,FreqDist,LidstoneProbDist,MLEProbDist
from nltk.tag.hmm import HiddenMarkovModelTagger

In [3]:
# Task 1. Load and explore your data
# 1). load train/test samples from Brown corpus files, brown-train.txt, brown-test.txt.
# 2). load all 12 tags from brown-tag.txt and print it out
# 3). counting how many sentences and words in both train and test datasets.
# 4). for each tag, counting how many words in train and test. e.g, tag1: [count_tr, count_te]

# Your code

## Task 1. Load and explore your data
### 1). load train/test samples from Brown corpus files, brown-train.txt, brown-test.txt.

In [4]:
def load_tag():
    return np.loadtxt('brown-tag.txt', skiprows=0, dtype=str)

def load_data(train=True):
    if train:
        path='brown-train.txt'
    else:
        path='brown-test.txt'
    data=[]
    sent_count=0
    # load txt
    with open(path, 'r') as f:
        for line in f:
            p = line.strip()
            if p == '':
                continue
            if '\t' not in p:
                sent_name=p
                sent_count+=1
                continue
            data.append(p.split('\t')+[sent_count-1, sent_name])
        data = pd.DataFrame(data,columns=['word', 'tag','sent_index','sent_name'])
    return data

In [5]:
trainset=load_data()
testset=load_data(train=False)

### 2). load all 12 tags from brown-tag.txt and print it out

In [6]:
tags=load_tag()

In [7]:
print('number of tags:',len(tags))
print(tags)

number of tags: 12
['.' 'ADJ' 'ADP' 'ADV' 'CONJ' 'DET' 'NOUN' 'NUM' 'PRON' 'PRT' 'VERB' 'X']


### 3). counting how many sentences and words in both train and test datasets.

In [8]:
print('number of sentences in TRAIN SET:',trainset.sent_index.max())
print('number of words in TRAIN SET:',len(trainset))
print('number of sentences in TEST SET:',testset.sent_index.max())
print('number of words in TEST SET:',len(testset))

number of sentences in TRAIN SET: 45799
number of words in TRAIN SET: 928327
number of sentences in TEST SET: 11539
number of words in TEST SET: 232865


### 4). for each tag, counting how many words in train and test. e.g, tag1: [count_tr, count_te]

In [9]:
tag_count_trainset=trainset['word'].groupby(trainset['tag']).count()
tag_count_testset=testset['word'].groupby(testset['tag']).count()
pd.DataFrame([tag_count_trainset,tag_count_testset],index=['trainset','testset'])

tag,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
trainset,117723,66985,115752,44765,30455,109418,220451,11921,39657,23889,146199,1112
testset,29842,16736,29014,11474,7696,27601,55107,2953,9677,5940,36551,274


In [10]:
# Task 2. Build a baseline method, namely, the most frequent tagger.
#     If you can recall, we introduced a strong baseline method (See Dan's book in Page 163.),
#     where we label each word by using the most frequent-used tag associated with it.
# 1). find the most frequent class label for each word in the training data.
#     For example, {tr_word_1:tag_1,tr_word_2:tag_2,...}
# 2). use your built method to predict tags for both train and test datasets.
#     You should print out two values: the accuracies of train and test samples.
#     You would expect that the accuracy on train will be > 9.0 (but never = 1.0) and higher than on test.

# Notice: since there are unkown words in test samples. 
#  Following ways could handle this (choose one or create your own): 
#  1). mark all words that appear only once in the data with a "UNK-x" tag
#  2). tag every out-of-vocabulary word with the majority tag among all training samples.
#  3). find more methods in https://github.com/Adamouization/POS-Tagging-and-Unknown-Words

# Your code

## Task 2. Build a baseline method, namely, the most frequent tagger.
     If you can recall, we introduced a strong baseline method (See Dan's book in Page 163.),
     where we label each word by using the most frequent-used tag associated with it.

### 1). find the most frequent class label for each word in the training data.
     For example, {tr_word_1:tag_1,tr_word_2:tag_2,...}

`Get the frequency of each tag of each word`

In [11]:
# get unique words
unque_words=pd.unique(trainset['word'])
word_count=pd.DataFrame(np.zeros((len(unque_words), len(tags))),columns=list(tags), index=unque_words)
# count frequency
for i in range(len(trainset)):
    word_count.loc[trainset.iloc[i]['word'],trainset.iloc[i]['tag']]+=1
word_count=word_count.astype(int)
word_count

,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
Mr.,0,0,0,0,0,0,696,0,0,0,0,0
Podger,0,0,0,0,0,0,19,0,0,0,0,0
had,0,0,0,0,0,0,0,0,0,0,4106,1
thanked,0,0,0,0,0,0,0,0,0,0,5,0
him,0,0,0,0,0,0,0,0,2060,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Mallory,0,0,0,0,0,0,1,0,0,0,0,0
Undertaken,0,0,0,0,0,0,0,0,0,0,1,0
chairmanship,0,0,0,0,0,0,1,0,0,0,0,0
Weigle,0,0,0,0,0,0,1,0,0,0,0,0


`Find the most frequency tag of each word`

In [12]:
word_count_most_freq=word_count.apply(pd.Series.idxmax,axis=1)
word_count_most_freq

Mr.             NOUN
Podger          NOUN
had             VERB
thanked         VERB
him             PRON
                ... 
Mallory         NOUN
Undertaken      VERB
chairmanship    NOUN
Weigle          NOUN
1946-52          NUM
Length: 50490, dtype: object

### 2). use your built method to predict tags for both train and test datasets.
     You should print out two values: the accuracies of train and test samples.
     You would expect that the accuracy on train will be > 9.0 (but never = 1.0) and higher than on test.

`Get the majority tag in train set`

In [13]:
majority_tag=tag_count_trainset.idxmax()
print('The majority tag in trainset is \'%s\''%majority_tag)

The majority tag in trainset is 'NOUN'


`Label each word by using the most frequent-used tag associated with it.`

`Tag every out-of-vocabulary word with the majority tag among all training samples. `

In [14]:
trainset['tag_baseline']=trainset['word'].apply(lambda x:word_count_most_freq[x] if x in word_count_most_freq else majority_tag)
testset['tag_baseline']=testset['word'].apply(lambda x:word_count_most_freq[x] if x in word_count_most_freq else majority_tag)

In [15]:
trainacc_baseline=(trainset['tag']==trainset['tag_baseline']).mean()
testacc_baseline=(testset['tag']==testset['tag_baseline']).mean()
print('Accuracy in TRAIN SET:',trainacc_baseline)
print('Accuracy in TEST SET:',testacc_baseline)

Accuracy in TRAIN SET: 0.9571961173164197
Accuracy in TEST SET: 0.945165653919653


In [16]:
# Task 3. Build an HMM tagger. 
# 1) You should use nltk.tag.HiddenMarkovModelTagger to build an HMM tagger.
#    It has parameters: symbols, states, transitions, outputs, priors, transform (ignore it).
#    Specify these parameters properly. For example, you can use MLE to estimate transitions, outputs and priors.
#    That is, MLE to estimate matrix A (transition matrix), and matrix B (output probabilites) (See. Page 8.4.3)
# 2) After build your model, report both the accuracy of HMM tagger for train samples and test samples.
# 
# 3) Compared with your baseline method, discuss that why your HMM tagger is better/worse than baseline method.

# Notice: You may also need to handle unknown words just like Task 2.

# Your code

## Task 3. Build an HMM tagger. 
### 1) You should use nltk.tag.HiddenMarkovModelTagger to build an HMM tagger.
    It has parameters: symbols, states, transitions, outputs, priors, transform (ignore it).
    Specify these parameters properly. For example, you can use MLE to estimate transitions, outputs and priors.
    That is, MLE to estimate matrix A (transition matrix), and matrix B (output probabilites) (See. Page 8.4.3)

`Get the proper format of labeled sequence and test sequence`

In [17]:
trainset['word,tag']=trainset[['word','tag']].apply(tuple,axis=1)
testset['word,tag']=testset[['word','tag']].apply(tuple,axis=1)
labeled_seq=trainset['word,tag'].groupby(trainset['sent_index']).agg(tuple).values.tolist()
test_seq=testset['word,tag'].groupby(testset['sent_index']).agg(tuple).values.tolist()

`Use this function to build my HMM model.`

`I can choose smooth=True/False to control whether to use probability distributions with smoothing.`

`With smoothing,we can handle the unseen word in testset properly.`

In [18]:
def get_hmm(smooth=True):
    # use LidstoneProbDist if we need smoothing else MLEProbDist
    if smooth:
        estimator = lambda fdist, bins: LidstoneProbDist(fdist, 0.1, bins)
    else:
        estimator = lambda fdist, bins: MLEProbDist(fdist)
    # get the unique word set
    symbols=trainset['word'].unique()
    priors = FreqDist()
    transitions = ConditionalFreqDist()
    outputs = ConditionalFreqDist()
    # count the frequency
    for sequence in labeled_seq:
        last = None
        for symbol,state in sequence:
            if last is None:
                priors[state] += 1
            else:
                transitions[last][state] += 1
            outputs[state][symbol] += 1
            last = state

    # create probability distributions
    priors = estimator(priors,len(tags))
    transitions = ConditionalProbDist(transitions, estimator, len(tags))
    outputs = ConditionalProbDist(outputs, estimator, len(symbols))
    return HiddenMarkovModelTagger(symbols, tags, transitions, outputs, priors)

`Build my HMM model with and without smoothing.`

In [19]:
model1=get_hmm()
model2=get_hmm(False)

### 2) After build your model, report both the accuracy of HMM tagger for train samples and test samples.

In [20]:
print('HMM with smoothing:')
print('TRAINSET',end=' ')
model1.test(labeled_seq)
print('TESTSET',end=' ')
model1.test(test_seq)
print('\nHMM without smoothing:')
print('TRAINSET',end=' ')
model2.test(labeled_seq)
print('TESTSET',end=' ')
model2.test(test_seq)

HMM with smoothing:
TRAINSET accuracy over 928327 tokens: 96.99
TESTSET accuracy over 232865 tokens: 95.10

HMM without smoothing:
TRAINSET accuracy over 928327 tokens: 97.56
TESTSET accuracy over 232865 tokens: 75.79


### 3) Compared with your baseline method, discuss that why your HMM tagger is better/worse than baseline method.

`We can see that HMM with smoothing is better than baseline both in TRAINSET and TESTSET. That is because HMM with smoothing use more complex model which takes the transition between states and the output probability into consider, and it also can handle a general case while it use smoothing to aviod unseen word.`

`HMM without smoothing is much better than baseline in TRAINSET but also much worse than baseline in TESTSET. That maybe results in its' not consider about the unseen words,which cause the overfitting.`